In [3]:
from pathlib import Path
import pandas as pd
import json

# 흐름 / 파일 경로 지정
data_dir = Path("../week04/data")
orders_path = data_dir / "claude_orders.csv"

# 조건 / 경로 존재하는지 확인
assert data_dir.exists(), "❌ data 폴더가 존재하지 않습니다"
assert orders_path.exists(), "❌ claude_orders.csv 파일이 없습니다"

# 1. 파일 로드
df = pd.read_csv(orders_path)

# 2. order_info 파싱
df["info"] = df["order_info"].apply(json.loads)

# 3. price, visit_count 추출해서 컬럼 추가
# → price_category, visit_grade 등등

# 4. groupby 실습
# → 나이대, 결제수단 등 기준으로 묶어서 집계


In [9]:
df["price_category"] = df["info"].apply(lambda x: "고가" if x.get("items",[{}])[0].get("price",0) >= 10000
                                        else "중가" if x.get("items",[{}])[0].get("price",0) >= 5000
                                        else "저가")

df["visit_grade"] = df["info"].apply(lambda x: "단골" if x.get("customer",{}).get("visit_count", 0) >= 15
                                     else "일반" if x.get("customer",{}).get("visit_count", 0) >= 5
                                     else "신규")

df["age_group"] = df["info"].apply(lambda x: x.get("customer", {}).get("age_group"))

df["payment_method"] = df["info"].apply(lambda x: x.get("payment", {}).get("method"))

In [8]:
# 조건 / 기준: 나이대(age_group)별
# 계산: 총 주문 금액의 평균

df.groupby("age_group")["total_amount"].mean()


age_group
20s    6500.0
30s    7250.0
40s    9600.0
Name: total_amount, dtype: float64

In [10]:
df.groupby("payment_method")["total_amount"].sum()


payment_method
card        22600
cash         4800
naverpay     9700
Name: total_amount, dtype: int64

In [11]:
df.groupby(["age_group", "payment_method"])["total_amount"].agg(["count", "sum", "mean"])


count    sum    mean
age_group payment_method                      
20s       card                2  13000  6500.0
30s       cash                1   4800  4800.0
          naverpay            1   9700  9700.0
40s       card                1   9600  9600.0

In [12]:
df.groupby("visit_grade")["total_amount"].sum()


visit_grade
단골    18100
신규     4500
일반    14500
Name: total_amount, dtype: int64

In [13]:
df.groupby(["price_category", "visit_grade"])["total_amount"].mean()

price_category  visit_grade
저가              단골             8500.0
                신규             4500.0
                일반             4800.0
중가              단골             9600.0
                일반             9700.0
Name: total_amount, dtype: float64

In [14]:
df.groupby(["price_category", "visit_grade"])["total_amount"].agg(["mean", "sum", "count"])


mean   sum  count
price_category visit_grade                     
저가             단골           8500.0  8500      1
               신규           4500.0  4500      1
               일반           4800.0  4800      1
중가             단골           9600.0  9600      1
               일반           9700.0  9700      1